In [1]:
import numpy as np
from scipy import stats, io, signal, linalg
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
from scipy.spatial.distance import squareform, pdist
from scipy.stats import kendalltau
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn import metrics
from nilearn.datasets.utils import _uncompress_file, _fetch_file
import warnings
from os.path import join
import os
from statsmodels.tsa.api import VAR
from idtxl.multivariate_mi import MultivariateMI
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from tigramite import data_processing as pp
from tigramite.independence_tests import ParCorr
from tigramite.pcmci import PCMCI

Could not import packages for CMIknn and GPDC estimation
Could not import packages for CMIknn and GPDC estimation


In [2]:
def normalize_0_mean_1_std(inp_series):
    inp_series=inp_series.copy()
    mean_ts=np.array([inp_series.mean(axis=1)]).transpose()
    mean_ts_mtrx = mean_ts*np.ones((1,inp_series.shape[1]));
    unb_data_mtrx = inp_series - mean_ts_mtrx
    p = np.power(unb_data_mtrx,2)
    s=np.array([p.sum(axis=1)]).transpose()
    sc=np.sqrt(s/p.shape[1])
    sc2=sc*(np.ones((1,p.shape[1])))
    nrm= np.divide(unb_data_mtrx,sc2)
    return nrm

In [3]:
def Clust_cent_not_separating_train_indices( outProj,param):
    data=outProj.copy()
    if len(data.shape)==1:
        data=data[np.newaxis,:]
    voxel_num, t = data.shape
    inputi=np.zeros((t-param['win'],voxel_num,param['win']))
    for ctrj in range(t-param['win']):
        inputi[ctrj,:,:]=data[:,ctrj:ctrj+param['win']]

    input=np.reshape(inputi,(t-param['win'],voxel_num*param['win']))
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=param['nCodes'], random_state=0).fit(input)
    return kmeans.cluster_centers_.transpose()

In [4]:
inp_series=np.array([[1,7,3,4,0],[1,2,4,0,2],[5,3,0,7,2],[2, 3, 5, 7, 0]])
inp_series=normalize_0_mean_1_std(inp_series)
inp_series

array([[-0.81649658,  1.63299316,  0.        ,  0.40824829, -1.22474487],
       [-0.60302269,  0.15075567,  1.6583124 , -1.35680105,  0.15075567],
       [ 0.66208471, -0.16552118, -1.40693001,  1.4896906 , -0.57932412],
       [-0.57932412, -0.16552118,  0.66208471,  1.4896906 , -1.40693001]])

In [5]:
y=inp_series
codes1=3
codes2=2
wi=2

data=y

outProj=data    #outProj is X  

full_size=data.shape[0]
it=1

param=dict()
param['training']=0.9
param['nCodes'] = codes1
param['fSigma'] =0.40
param['win']=wi
param['ite']=it

T = outProj.shape[1] # number of time observations
tau=1

num_vecs=(T-(param['win']-1)*tau)-1;


outputi = data[:,param['win']:].transpose()
ClusterCtrPCA =  Clust_cent_not_separating_train_indices( outProj,param )

if len(ClusterCtrPCA.shape)==2:
    ClusterCtrPCA=ClusterCtrPCA[:,:,np.newaxis]

ClusterPCAfull=ClusterCtrPCA[:,:,0]  # one run of the iteration used only to get the cluster centers

param1=param
param1['nCodes']=codes2
clus_series=np.zeros((full_size,param['win'],param1['nCodes']))


# size of clus_series = length of series, size of input to the network, number of codes 
for m in range(full_size):
    clus_series[m,:,:] = Clust_cent_not_separating_train_indices(data[m,:],param1) 
    
# one run of the iteration used only to get the cluster centers

# Get activities for both training and testing for individual series
par=param1
curr_act_train=np.zeros((full_size,1,num_vecs,param1['nCodes']))

for m in range(full_size):
    ClusterCtr=np.squeeze(clus_series[m])
    inp=data[m,:]
#     curr_act_train(m,:,:,:,:) = grbf_activity_full_no_sep_train( inp,ClusterCtr,par,generalize );


In [13]:
generalize=1

In [7]:
inp_series=np.array([[1,7,3,4,0],[1,2,4,0,2],[5,3,0,7,2],[2, 3, 5, 7, 0]])

Clust_cent_not_separating_train_indices(inp_series[1,:], param)

array([[1.5, 4. ],
       [3. , 0. ]])